# Artificial Neural Networks (CS-456)
## Miniproject 1: Tic Tac Toe
- Mickaël Achkar (322190)
- Yehya El Hassan (325932)


In [1]:
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer

In [ ]:
environment = TictactoeEnv()

Implementing a Tic Tac Toc Player using Q-learning. With that, we will create a Q-table representing all the states and actions possible and we will progressively update the values in the table. The possible number of state is $5478$ and the total possible number of actions at each state is upper bounded by $9$. 

In [ ]:
#Calculate the total number of states and actions for Tic-Tac-Toe
number_of_states = 5478
number_of_actions = 9

#Initialize the Q-table
Q_table = np.zeros((number_of_states, number_of_actions))

#RL Hyper-params
number_of_episodes = 20000
epsilon = 0.1
learning_rate = 0.05
discount_factor = 0.99